In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from apify_client import ApifyClient
import json

def collect_data(cities):
    client = ApifyClient(
        token='apify_api_BdgKVIBQfeQk9rBpUdoOlNhBG7484q15CTd0')
    input_data = {
                "searchStringsArray": cities,
                "maxCrawledPlacesPerSearch": 50,
                "language": "en",
                "maxImages": 10,
                "onlyDataFromSearchPage": False,
                "maxReviews": 50,
                "scrapeReviewerName": True,
                "scrapeReviewerId": False,
                "scrapeReviewerUrl": True,
                "scrapeReviewId": True,
                "scrapeReviewUrl": True,
                "scrapeResponseFromOwnerText": False,
                "oneReviewPerRow": False,
                "reviewsSort": "newest",
                "allPlacesNoSearchAction": ""
                }
    # print("Before client.actor")
    run = client.actor(
        "compass/crawler-google-places").call(run_input=input_data)
    # print("After client.actor")
    dataset = client.dataset(run['defaultDatasetId'])
    hotels = dataset.list_items().items
    file_path = "data/raw_data.json"
    with open(file_path, "w") as json_file:
        json.dump(hotels, json_file)
    return hotels

In [3]:
cities = [ "hotels and resorts in Cape Coast, Ghana",
            "hotels and resorts in West Region, Ghana",
            "hotels and resorts in Volta Region, Ghana",
            "hotels and resorts in Accra"]

In [4]:
# collect_data(cities)

In [5]:
# collect_data(["hotels and resorts in Eastern Region, Ghana"])

In [6]:
metric_cols = ["title", "city", "location", "countryCode", "placeId", "reviewsCount", "totalScore", "hotelStars", "additionalInfo", "price" , "categoryName", "reviewsTags"]
amenities = [""]
review_cols =["placeId", "reviews"]
review_details = ["publishedAtDate", "text", "stars"]
info_cols = ["placeId", "title","city", "phone", "imageUrls", "website", "additionalInfo", "location", "categoryName"]

In [7]:
# metrics_data =[]
# reviews_data = []
# info_data = []
file_path = "../data/hotels-data.json"

def fill_metrics(col_list):
    with open(file_path,  encoding="utf-8") as json_file:
        json_text = json.load(json_file)
        metrics_data ={}
        for col in col_list:
            col_data = [item[col] for item in json_text]
            metrics_data[col] = col_data

    return metrics_data

In [8]:
metrics_data = fill_metrics(metric_cols)

In [9]:
metrics_table = pd.DataFrame(metrics_data)
metrics_table.head()

,title,city,location,countryCode,placeId,reviewsCount,totalScore,hotelStars,additionalInfo,price,categoryName,reviewsTags
0,Jonat Hotel,Cape Coast,"{'lat': 5.1326868, 'lng': -1.2707224}",GH,ChIJhd6ATRj_3Q8R7nqmtm6INVI,6,4.7,None,{},None,Hotel,[]
1,Drowaa Plus Hotel Annex,Cape Coast,"{'lat': 5.1187098, 'lng': -1.256577}",GH,ChIJU9N0DWz_3Q8Rg0eDgnarV8Y,12,4.2,None,{},None,Hotel,[]
2,Sanna Lodge,Cape Coast,"{'lat': 5.1036076, 'lng': -1.2703398}",GH,ChIJA0RtJd7-3Q8Rp9cBaQMA_9o,10,3.6,None,{'Amenities': [{'Kid-friendly': True}]},None,Lodge,[]
3,Fantas Folly,Takoradi,"{'lat': 4.8294648, 'lng': -1.8985344}",GH,ChIJA2dVyhKc5w8RzaeO_CSow8E,145,4.3,None,{'Amenities': [{'Kid-friendly': True}]},None,Hotel,"[{'title': 'beach', 'count': 38}, {'title': 'r..."
4,Ankobra Bridge Hotel and Resort,Asanta,"{'lat': 4.901078, 'lng': -2.2673678}",GH,ChIJL7Hywp7x5w8RFvPm-57fTwc,44,4.4,None,{},None,Hotel,"[{'title': 'service', 'count': 4}, {'title': '..."


In [10]:

def fill_reviews():
    with open(file_path,  encoding="utf-8") as json_file:
        data = json.load(json_file)
    reviews_data = []
    for item in data:
        for review in item['reviews']:
            extracted_data = {
                'hotel_id': item['placeId'],
                'date': review['publishedAtDate'],
                'rating': review['stars'],
                'comment': review['text']
                # Add more key-value pairs as needed
            }
            reviews_data.append(extracted_data)

    df = pd.DataFrame(reviews_data)
    return df

In [11]:
reviews_table = fill_reviews()

In [12]:
reviews_table.head()

,hotel_id,date,rating,comment
0,ChIJhd6ATRj_3Q8R7nqmtm6INVI,2023-05-01T08:53:17.543Z,4.0,None
1,ChIJhd6ATRj_3Q8R7nqmtm6INVI,2023-02-10T09:25:21.640Z,5.0,I had a wonderful experience at Jonat Hotel. T...
2,ChIJhd6ATRj_3Q8R7nqmtm6INVI,2023-01-23T16:02:18.862Z,5.0,I enjoyed my stay here. For a serene environme...
3,ChIJhd6ATRj_3Q8R7nqmtm6INVI,2023-01-23T07:27:23.598Z,5.0,"The couple of times I have been there, were ve..."
4,ChIJhd6ATRj_3Q8R7nqmtm6INVI,2023-01-23T06:19:51.128Z,5.0,Everything there was nice and super clean 👍👍👍


In [13]:
reviews_table.to_csv('../data/raw_reviews.csv')

In [14]:
info_table = fill_metrics(info_cols)

In [15]:
info_table = pd.DataFrame(info_table)

In [16]:
info_table.head()

,placeId,title,city,phone,imageUrls,website,additionalInfo,location,categoryName
0,ChIJhd6ATRj_3Q8R7nqmtm6INVI,Jonat Hotel,Cape Coast,+233 20 421 7037,[https://lh5.googleusercontent.com/p/AF1QipO-l...,None,{},"{'lat': 5.1326868, 'lng': -1.2707224}",Hotel
1,ChIJU9N0DWz_3Q8Rg0eDgnarV8Y,Drowaa Plus Hotel Annex,Cape Coast,+233 50 009 6604,[https://lh5.googleusercontent.com/p/AF1QipO_T...,https://drowaa-plus-hotel-annex.business.site/...,{},"{'lat': 5.1187098, 'lng': -1.256577}",Hotel
2,ChIJA0RtJd7-3Q8Rp9cBaQMA_9o,Sanna Lodge,Cape Coast,None,[https://lh5.googleusercontent.com/p/AF1QipPP0...,None,{'Amenities': [{'Kid-friendly': True}]},"{'lat': 5.1036076, 'lng': -1.2703398}",Lodge
3,ChIJA2dVyhKc5w8RzaeO_CSow8E,Fantas Folly,Takoradi,+233 24 321 3677,[https://lh5.googleusercontent.com/p/AF1QipPOh...,http://www.fantasfolly.com/,{'Amenities': [{'Kid-friendly': True}]},"{'lat': 4.8294648, 'lng': -1.8985344}",Hotel
4,ChIJL7Hywp7x5w8RFvPm-57fTwc,Ankobra Bridge Hotel and Resort,Asanta,+233 33 209 7319,[https://lh5.googleusercontent.com/p/AF1QipNk0...,https://ankobrabridgehotelandresort.com/,{},"{'lat': 4.901078, 'lng': -2.2673678}",Hotel


In [17]:
metrics_table['city'].value_counts()

city
Accra                                  50
Cape Coast                             48
Ho                                     13
Takoradi                               13
Sogakope                                7
Keta                                    5
Hohoe                                   5
Elmina                                  5
Ada Foah                                3
Axim                                    3
Esiama                                  3
Akwidaa                                 2
Busua                                   2
Shama                                   2
Sogakofe                                1
Kpando                                  1
Biakpa                                  1
Asanta                                  1
Mafi - Adidome                          1
Dzodze                                  1
Busua Beach                             1
Beach                                   1
Akosombo                                1
Sokode Etoe                  

In [18]:
file_path = "../data/east-hotels.json"
def fill_reviews():
    with open(file_path,  encoding="utf-8") as json_file:
        data = json.load(json_file)
    reviews_data = []
    for item in data:
        for review in item['reviews']:
            extracted_data = {
                'hotel_id': item['placeId'],
                'date': review['publishedAtDate'],
                'rating': review['stars'],
                'comment': review['text']
                # Add more key-value pairs as needed
            }
            reviews_data.append(extracted_data)

    df = pd.DataFrame(reviews_data)
    return df

In [19]:
east_reviews = fill_reviews()

In [20]:
east_reviews.head()

,hotel_id,date,rating,comment
0,ChIJeTnHxwVx3w8RPL5k90qYuyI,2023-05-18T13:40:54.957Z,5,I'd want to do this again
1,ChIJeTnHxwVx3w8RPL5k90qYuyI,2023-05-17T13:48:56.931Z,5,Let me relax and talk about this amazing place...
2,ChIJeTnHxwVx3w8RPL5k90qYuyI,2023-05-17T05:54:05.175Z,5,Amazing pool great food
3,ChIJeTnHxwVx3w8RPL5k90qYuyI,2023-05-14T11:25:26.644Z,5,None
4,ChIJeTnHxwVx3w8RPL5k90qYuyI,2023-05-11T16:00:28.382Z,5,None


In [21]:
east_metrics = fill_metrics(metric_cols)
east_metrics = pd.DataFrame(east_metrics)
east_metrics.head()

,title,city,location,countryCode,placeId,reviewsCount,totalScore,hotelStars,additionalInfo,price,categoryName,reviewsTags
0,Hillburi,Aburi,"{'lat': 5.8276147, 'lng': -0.1844512}",GH,ChIJeTnHxwVx3w8RPL5k90qYuyI,1257,4.2,None,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...",$125,Hotel,"[{'title': 'environment', 'count': 82}, {'titl..."
1,The FLOAT,Akosombo,"{'lat': 6.2980265, 'lng': 0.03101}",GH,ChIJ7XeQ0OGpIBAR7kupYMNEPSA,172,4.5,None,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...",None,Resort hotel,"[{'title': 'environment', 'count': 13}, {'titl..."
2,Beige Village Golf Resort & Spa,New Abirem,"{'lat': 6.3397819, 'lng': -0.9975905}",GH,ChIJ6xZ_ouuz3g8REI-EZRT0DEQ,46,4.0,None,{},None,Serviced apartment,"[{'title': 'environment', 'count': 4}]"
3,Aylos Bay Garden Restaurant & Lodge,Atimpoku,"{'lat': 6.2439045, 'lng': 0.0921707}",GH,ChIJV5EB4l2vIBARXH97YYr0Ygo,344,4.0,None,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...",None,Resort hotel,"[{'title': 'environment', 'count': 21}, {'titl..."
4,"Blue Royal Hotel, Larteh",Larteh,"{'lat': 5.9373755, 'lng': -0.0643552}",GH,ChIJpydwbHdj3w8RM4Y8w52voM0,483,3.9,3-star hotel,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...",None,Hotel,"[{'title': 'environment', 'count': 34}, {'titl..."


In [22]:
east_info = fill_metrics(info_cols)
east_info = pd.DataFrame(east_info)
east_info.head()

,placeId,title,city,phone,imageUrls,website,additionalInfo,location,categoryName
0,ChIJeTnHxwVx3w8RPL5k90qYuyI,Hillburi,Aburi,+233 55 252 6303,[https://lh5.googleusercontent.com/p/AF1QipPX3...,http://www.hillburi.com/,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...","{'lat': 5.8276147, 'lng': -0.1844512}",Hotel
1,ChIJ7XeQ0OGpIBAR7kupYMNEPSA,The FLOAT,Akosombo,+233 20 210 9793,[https://lh5.googleusercontent.com/p/AF1QipNdY...,https://thefloatgh.com/,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...","{'lat': 6.2980265, 'lng': 0.03101}",Resort hotel
2,ChIJ6xZ_ouuz3g8REI-EZRT0DEQ,Beige Village Golf Resort & Spa,New Abirem,+233 30 298 0193,[https://lh3.googleusercontent.com/gps-proxy/A...,None,{},"{'lat': 6.3397819, 'lng': -0.9975905}",Serviced apartment
3,ChIJV5EB4l2vIBARXH97YYr0Ygo,Aylos Bay Garden Restaurant & Lodge,Atimpoku,+233 24 337 4443,[https://lh5.googleusercontent.com/p/AF1QipNwW...,http://www.aylosbayghana.com/,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...","{'lat': 6.2439045, 'lng': 0.0921707}",Resort hotel
4,ChIJpydwbHdj3w8RM4Y8w52voM0,"Blue Royal Hotel, Larteh",Larteh,+233 54 551 8899,[https://lh5.googleusercontent.com/p/AF1QipMZn...,http://blueroyalhotels.com/,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...","{'lat': 5.9373755, 'lng': -0.0643552}",Hotel


In [23]:
hotel_info = pd.concat([info_table, east_info], axis=0)
hotel_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 0 to 49
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   placeId         250 non-null    object
 1   title           250 non-null    object
 2   city            240 non-null    object
 3   phone           223 non-null    object
 4   imageUrls       250 non-null    object
 5   website         121 non-null    object
 6   additionalInfo  250 non-null    object
 7   location        250 non-null    object
 8   categoryName    250 non-null    object
dtypes: object(9)
memory usage: 19.5+ KB


In [24]:
hotel_metrics = pd.concat([metrics_table, east_metrics], axis=0)
hotel_metrics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 0 to 49
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           250 non-null    object 
 1   city            240 non-null    object 
 2   location        250 non-null    object 
 3   countryCode     250 non-null    object 
 4   placeId         250 non-null    object 
 5   reviewsCount    250 non-null    int64  
 6   totalScore      240 non-null    float64
 7   hotelStars      72 non-null     object 
 8   additionalInfo  250 non-null    object 
 9   price           68 non-null     object 
 10  categoryName    250 non-null    object 
 11  reviewsTags     250 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 25.4+ KB


In [25]:
hotel_reviews = pd.concat([reviews_table, east_reviews], axis = 0)
hotel_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9347 entries, 0 to 1981
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   hotel_id  9347 non-null   object 
 1   date      9347 non-null   object 
 2   rating    9342 non-null   float64
 3   comment   4659 non-null   object 
dtypes: float64(1), object(3)
memory usage: 365.1+ KB


In [26]:
hotel_metrics["reviewsTags"]

0                                                    []
1                                                    []
2                                                    []
3     [{'title': 'beach', 'count': 38}, {'title': 'r...
4     [{'title': 'service', 'count': 4}, {'title': '...
                            ...                        
45    [{'title': 'environment', 'count': 19}, {'titl...
46    [{'title': 'environment', 'count': 22}, {'titl...
47    [{'title': 'environment', 'count': 21}, {'titl...
48    [{'title': 'environment', 'count': 35}, {'titl...
49    [{'title': 'environment', 'count': 243}, {'tit...
Name: reviewsTags, Length: 250, dtype: object

In [27]:
def check_beach(row, key, value):
    for d in row:
        if key in d and d[key] == value:
            return True
    return False

key = 'title'
value = 'beach'
hotel_metrics['beach'] = hotel_metrics['reviewsTags'].apply(lambda row: {key: value} if check_beach(row, key, value) else {})

In [28]:
hotel_metrics.head()

,title,city,location,countryCode,placeId,reviewsCount,totalScore,hotelStars,additionalInfo,price,categoryName,reviewsTags,beach
0,Jonat Hotel,Cape Coast,"{'lat': 5.1326868, 'lng': -1.2707224}",GH,ChIJhd6ATRj_3Q8R7nqmtm6INVI,6,4.7,None,{},None,Hotel,[],{}
1,Drowaa Plus Hotel Annex,Cape Coast,"{'lat': 5.1187098, 'lng': -1.256577}",GH,ChIJU9N0DWz_3Q8Rg0eDgnarV8Y,12,4.2,None,{},None,Hotel,[],{}
2,Sanna Lodge,Cape Coast,"{'lat': 5.1036076, 'lng': -1.2703398}",GH,ChIJA0RtJd7-3Q8Rp9cBaQMA_9o,10,3.6,None,{'Amenities': [{'Kid-friendly': True}]},None,Lodge,[],{}
3,Fantas Folly,Takoradi,"{'lat': 4.8294648, 'lng': -1.8985344}",GH,ChIJA2dVyhKc5w8RzaeO_CSow8E,145,4.3,None,{'Amenities': [{'Kid-friendly': True}]},None,Hotel,"[{'title': 'beach', 'count': 38}, {'title': 'r...",{'title': 'beach'}
4,Ankobra Bridge Hotel and Resort,Asanta,"{'lat': 4.901078, 'lng': -2.2673678}",GH,ChIJL7Hywp7x5w8RFvPm-57fTwc,44,4.4,None,{},None,Hotel,"[{'title': 'service', 'count': 4}, {'title': '...",{}


In [29]:
hotel_metrics["additionalInfo"]

0                                                    {}
1                                                    {}
2               {'Amenities': [{'Kid-friendly': True}]}
3               {'Amenities': [{'Kid-friendly': True}]}
4                                                    {}
                            ...                        
45    {'Accessibility': [{'Wheelchair accessible ent...
46    {'Amenities': [{'Free Wi-Fi': True}, {'Free br...
47                                                   {}
48    {'Amenities': [{'Free Wi-Fi': True}, {'Free br...
49    {'Amenities': [{'Free Wi-Fi': True}, {'Free br...
Name: additionalInfo, Length: 250, dtype: object

In [30]:

def extract_keys(row, key):
    amenities ={}
    keys =[]
    if key in row:
        for d in row[key]:
            keys.extend(list(d.keys()))
        amenities["amenities"] = keys
        return amenities
    return {}     


In [31]:
key ='Amenities'
hotel_metrics["experience"] = hotel_metrics["additionalInfo"].apply(lambda row: extract_keys(row, key))

In [32]:
hotel_metrics.head()

,title,city,location,countryCode,placeId,reviewsCount,totalScore,hotelStars,additionalInfo,price,categoryName,reviewsTags,beach,experience
0,Jonat Hotel,Cape Coast,"{'lat': 5.1326868, 'lng': -1.2707224}",GH,ChIJhd6ATRj_3Q8R7nqmtm6INVI,6,4.7,None,{},None,Hotel,[],{},{}
1,Drowaa Plus Hotel Annex,Cape Coast,"{'lat': 5.1187098, 'lng': -1.256577}",GH,ChIJU9N0DWz_3Q8Rg0eDgnarV8Y,12,4.2,None,{},None,Hotel,[],{},{}
2,Sanna Lodge,Cape Coast,"{'lat': 5.1036076, 'lng': -1.2703398}",GH,ChIJA0RtJd7-3Q8Rp9cBaQMA_9o,10,3.6,None,{'Amenities': [{'Kid-friendly': True}]},None,Lodge,[],{},{'amenities': ['Kid-friendly']}
3,Fantas Folly,Takoradi,"{'lat': 4.8294648, 'lng': -1.8985344}",GH,ChIJA2dVyhKc5w8RzaeO_CSow8E,145,4.3,None,{'Amenities': [{'Kid-friendly': True}]},None,Hotel,"[{'title': 'beach', 'count': 38}, {'title': 'r...",{'title': 'beach'},{'amenities': ['Kid-friendly']}
4,Ankobra Bridge Hotel and Resort,Asanta,"{'lat': 4.901078, 'lng': -2.2673678}",GH,ChIJL7Hywp7x5w8RFvPm-57fTwc,44,4.4,None,{},None,Hotel,"[{'title': 'service', 'count': 4}, {'title': '...",{},{}


In [33]:
hotel_metrics.tail()

,title,city,location,countryCode,placeId,reviewsCount,totalScore,hotelStars,additionalInfo,price,categoryName,reviewsTags,beach,experience
45,BEIGE Village Golf Resort & Spa,New Abirem,"{'lat': 6.3462406, 'lng': -1.0017878}",GH,ChIJ_____-uz3g8R1G1_rRV22Qs,381,3.9,None,{'Accessibility': [{'Wheelchair accessible ent...,$62,Golf course,"[{'title': 'environment', 'count': 19}, {'titl...",{},{'amenities': ['Kid-friendly']}
46,"The Lansdown Resort, Aburi-Ghana",Aburi,"{'lat': 5.8058686, 'lng': -0.1969076}",GH,ChIJD2Plzc923w8RdAQWXdXjYzk,303,4.2,3-star hotel,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...",None,Resort hotel,"[{'title': 'environment', 'count': 22}, {'titl...",{},"{'amenities': ['Free Wi-Fi', 'Free breakfast',..."
47,BridgeView Resort,Akosombo,"{'lat': 6.2384595, 'lng': 0.0912979}",GH,ChIJFTAN5ImvIBARJ1BJu04amhA,713,4.5,None,{},$177,Resort hotel,"[{'title': 'environment', 'count': 21}, {'titl...",{},{}
48,Cactus Creek Hotel Ghana,Aburi,"{'lat': 5.844214, 'lng': -0.1650455}",GH,ChIJpZxowXpx3w8RsyC642zhEjk,296,4.3,None,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...",$84,Hotel,"[{'title': 'environment', 'count': 35}, {'titl...",{},"{'amenities': ['Free Wi-Fi', 'Free breakfast',..."
49,The Royal Senchi Resort/Hotel,Akosombo,"{'lat': 6.2195254, 'lng': 0.0891466}",GH,ChIJddSnqvGoIBAR2m3ekWBbs2o,3432,4.5,4-star hotel,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...",$245,Resort hotel,"[{'title': 'environment', 'count': 243}, {'tit...",{},"{'amenities': ['Free Wi-Fi', 'Free breakfast',..."


In [34]:
hotel_metrics = hotel_metrics.reset_index(drop =True)

In [35]:
hotel_metrics.tail()

,title,city,location,countryCode,placeId,reviewsCount,totalScore,hotelStars,additionalInfo,price,categoryName,reviewsTags,beach,experience
245,BEIGE Village Golf Resort & Spa,New Abirem,"{'lat': 6.3462406, 'lng': -1.0017878}",GH,ChIJ_____-uz3g8R1G1_rRV22Qs,381,3.9,None,{'Accessibility': [{'Wheelchair accessible ent...,$62,Golf course,"[{'title': 'environment', 'count': 19}, {'titl...",{},{'amenities': ['Kid-friendly']}
246,"The Lansdown Resort, Aburi-Ghana",Aburi,"{'lat': 5.8058686, 'lng': -0.1969076}",GH,ChIJD2Plzc923w8RdAQWXdXjYzk,303,4.2,3-star hotel,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...",None,Resort hotel,"[{'title': 'environment', 'count': 22}, {'titl...",{},"{'amenities': ['Free Wi-Fi', 'Free breakfast',..."
247,BridgeView Resort,Akosombo,"{'lat': 6.2384595, 'lng': 0.0912979}",GH,ChIJFTAN5ImvIBARJ1BJu04amhA,713,4.5,None,{},$177,Resort hotel,"[{'title': 'environment', 'count': 21}, {'titl...",{},{}
248,Cactus Creek Hotel Ghana,Aburi,"{'lat': 5.844214, 'lng': -0.1650455}",GH,ChIJpZxowXpx3w8RsyC642zhEjk,296,4.3,None,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...",$84,Hotel,"[{'title': 'environment', 'count': 35}, {'titl...",{},"{'amenities': ['Free Wi-Fi', 'Free breakfast',..."
249,The Royal Senchi Resort/Hotel,Akosombo,"{'lat': 6.2195254, 'lng': 0.0891466}",GH,ChIJddSnqvGoIBAR2m3ekWBbs2o,3432,4.5,4-star hotel,"{'Amenities': [{'Free Wi-Fi': True}, {'Free br...",$245,Resort hotel,"[{'title': 'environment', 'count': 243}, {'tit...",{},"{'amenities': ['Free Wi-Fi', 'Free breakfast',..."


In [36]:
def update_experiences(row, key):
    if key in row['beach']:
        row['experience']['amenities'].append('Beach')
        return row['experience']
    return row['experience']

In [37]:
key = 'title'
hotel_metrics['experience'] = hotel_metrics.apply(lambda row: update_experiences(row, key), axis=1)

In [38]:
hotel_metrics.head()

,title,city,location,countryCode,placeId,reviewsCount,totalScore,hotelStars,additionalInfo,price,categoryName,reviewsTags,beach,experience
0,Jonat Hotel,Cape Coast,"{'lat': 5.1326868, 'lng': -1.2707224}",GH,ChIJhd6ATRj_3Q8R7nqmtm6INVI,6,4.7,None,{},None,Hotel,[],{},{}
1,Drowaa Plus Hotel Annex,Cape Coast,"{'lat': 5.1187098, 'lng': -1.256577}",GH,ChIJU9N0DWz_3Q8Rg0eDgnarV8Y,12,4.2,None,{},None,Hotel,[],{},{}
2,Sanna Lodge,Cape Coast,"{'lat': 5.1036076, 'lng': -1.2703398}",GH,ChIJA0RtJd7-3Q8Rp9cBaQMA_9o,10,3.6,None,{'Amenities': [{'Kid-friendly': True}]},None,Lodge,[],{},{'amenities': ['Kid-friendly']}
3,Fantas Folly,Takoradi,"{'lat': 4.8294648, 'lng': -1.8985344}",GH,ChIJA2dVyhKc5w8RzaeO_CSow8E,145,4.3,None,{'Amenities': [{'Kid-friendly': True}]},None,Hotel,"[{'title': 'beach', 'count': 38}, {'title': 'r...",{'title': 'beach'},"{'amenities': ['Kid-friendly', 'Beach']}"
4,Ankobra Bridge Hotel and Resort,Asanta,"{'lat': 4.901078, 'lng': -2.2673678}",GH,ChIJL7Hywp7x5w8RFvPm-57fTwc,44,4.4,None,{},None,Hotel,"[{'title': 'service', 'count': 4}, {'title': '...",{},{}


In [39]:
hotel_metrics['latitude'] = hotel_metrics['location'].apply(lambda x: x['lat'])
hotel_metrics['longitude'] = hotel_metrics['location'].apply(lambda x: x['lng'])

In [40]:
hotel_metrics.head()

,title,city,location,countryCode,placeId,reviewsCount,totalScore,hotelStars,additionalInfo,price,categoryName,reviewsTags,beach,experience,latitude,longitude
0,Jonat Hotel,Cape Coast,"{'lat': 5.1326868, 'lng': -1.2707224}",GH,ChIJhd6ATRj_3Q8R7nqmtm6INVI,6,4.7,None,{},None,Hotel,[],{},{},5.132687,-1.270722
1,Drowaa Plus Hotel Annex,Cape Coast,"{'lat': 5.1187098, 'lng': -1.256577}",GH,ChIJU9N0DWz_3Q8Rg0eDgnarV8Y,12,4.2,None,{},None,Hotel,[],{},{},5.118710,-1.256577
2,Sanna Lodge,Cape Coast,"{'lat': 5.1036076, 'lng': -1.2703398}",GH,ChIJA0RtJd7-3Q8Rp9cBaQMA_9o,10,3.6,None,{'Amenities': [{'Kid-friendly': True}]},None,Lodge,[],{},{'amenities': ['Kid-friendly']},5.103608,-1.270340
3,Fantas Folly,Takoradi,"{'lat': 4.8294648, 'lng': -1.8985344}",GH,ChIJA2dVyhKc5w8RzaeO_CSow8E,145,4.3,None,{'Amenities': [{'Kid-friendly': True}]},None,Hotel,"[{'title': 'beach', 'count': 38}, {'title': 'r...",{'title': 'beach'},"{'amenities': ['Kid-friendly', 'Beach']}",4.829465,-1.898534
4,Ankobra Bridge Hotel and Resort,Asanta,"{'lat': 4.901078, 'lng': -2.2673678}",GH,ChIJL7Hywp7x5w8RFvPm-57fTwc,44,4.4,None,{},None,Hotel,"[{'title': 'service', 'count': 4}, {'title': '...",{},{},4.901078,-2.267368


In [41]:
remove_cols =["city", "title", "location", "countryCode", "categoryName", "reviewsTags", "beach", "additionalInfo"]
hotel_metrics = hotel_metrics.drop(remove_cols, axis = 1)

In [42]:
hotel_metrics.head()

,placeId,reviewsCount,totalScore,hotelStars,price,experience,latitude,longitude
0,ChIJhd6ATRj_3Q8R7nqmtm6INVI,6,4.7,None,None,{},5.132687,-1.270722
1,ChIJU9N0DWz_3Q8Rg0eDgnarV8Y,12,4.2,None,None,{},5.118710,-1.256577
2,ChIJA0RtJd7-3Q8Rp9cBaQMA_9o,10,3.6,None,None,{'amenities': ['Kid-friendly']},5.103608,-1.270340
3,ChIJA2dVyhKc5w8RzaeO_CSow8E,145,4.3,None,None,"{'amenities': ['Kid-friendly', 'Beach']}",4.829465,-1.898534
4,ChIJL7Hywp7x5w8RFvPm-57fTwc,44,4.4,None,None,{},4.901078,-2.267368


# INSERTING DATA TO DATABASE

In [43]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="jossytesting"
)


In [44]:
cursor = conn.cursor()
create_database_query = "CREATE DATABASE hotels_database"
cursor.execute(create_database_query)


DatabaseError: 1007 (HY000): Can't create database 'hotels_database'; database exists

In [ ]:
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="jossytesting",
    database = "hotels_database"
)

In [ ]:
cursor2 = connection.cursor()

In [ ]:
create_table_query = """
CREATE TABLE hotels (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    price DECIMAL(10, 2),
    category VARCHAR(50)
);
"""
cursor2.execute(create_table_query)
